# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 7

### Modelado de tópicos - LSI/LDA

#### **Nombres y matrículas de los integrantes del equipo:**



*   A01795061 - Ricardo Burboa Astorga
*   A01795353 - Alejandro Calderón Aguilar
*   A00278401 - Luis Antonio Calderón Mata
*   A01794243 - Erick Alexei Cambray Servín
*   A01795086 - Juan Carlos Campero Villa



In [1]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:

import pandas as pd
import json

import pandas as pd
import json
import re
import nltk
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SpanishStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

import multiprocessing

Using pip install fasttext-wheel instead solved the problem for me.

##**Pregunta - 1:**



1. Descarga el archivo noticiasTopicModeling.txt que se encuentra en Canvas. Este archivo consiste en
5658 noticias de varios periódicos de España. El archivo de texto es una lista en el siguiente
formato:
 [{“titular”:”Encabezado“, “texto”:”Cuerpo”}, … , {“titular”:”Encabezado”,”texto”:”Cuerpo”}]
Donde “titular” es el encabezado de la noticia y “texto” es el cuerpo del texto de dicha noticia. En
particular en esta actividad trabajarás solamente con los cuerpos de las noticias, sin incluir los
encabezados. Carga dicho archivo y genera un DataFrame de Pandas llamado “df” y que contiene
una única columna llamada “noticia” con 5658 renglones formados por los cuerpos de las noticias. 


In [2]:
DATA_PATH = r'Topic Modeling Activity.txt'

In [3]:
# Leer el archivo
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    data = file.read()

# Convertir el contenido del archivo a una lista de diccionarios
noticias = json.loads(data)

# Extraer los cuerpos de las noticias
cuerpos = [noticia['texto'] for noticia in noticias]

# Crear el DataFrame
df = pd.DataFrame(cuerpos, columns=['noticia'])

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())


                                             noticia
0  "España ha dejado de ser católica", decía Manu...
1  El clima de crispación social en Ceuta ha lleg...
2  El Gobierno ha alegado la suspensión de plazos...
3  Puedes mandar tu pregunta, sugerencia o queja ...
4  Panamá debe entregar esta semana a la Corte In...


##**Pregunta - 2:**



Realiza un proceso de limpieza. Aplica el preprocesamiento que consideres adecuado para texto en
español. Recuerda que el objetivo es identificar los tokens (palabras) que describan mejor la
distribución de cada tema.
NOTA: Recuerda que esta es una técnica no supervisada, por lo que no requerimos hacer una
partición de los datos.

In [4]:
import spacy
import re
import nltk
from nltk.corpus import stopwords

# Cargar modelo de spaCy para el español
nlp = spacy.load('es_core_news_sm')

# Descargar y configurar stopwords adicionales de NLTK en español
nltk.download('stopwords')
stop_words_spacy = nlp.Defaults.stop_words
stop_words_nltk = set(stopwords.words('spanish'))
combined_stopwords = stop_words_spacy.union(stop_words_nltk)


def limpiar_texto(texto):
    """
    Función para limpiar y preprocesar texto en español:
    - Convierte el texto a minúsculas para homogeneizar la capitalización.
    - Elimina caracteres no alfabéticos y números para obtener solo palabras.
    - Utiliza spaCy para analizar el texto y extraer tokens válidos.
    - Filtra los tokens eliminando stopwords y conservando solo palabras alfabéticas.
    - Devuelve el texto limpio como una cadena de texto con palabras separadas por espacio.
    """
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[\W_]+', ' ', texto)
    texto = re.sub(r'\d+', '', texto)
    # Procesar texto con spaCy
    doc = nlp(texto)
    # Filtrar tokens
    tokens = [
        token.text for token in doc
        if token.text not in combined_stopwords and token.is_alpha
    ]
    # Unir tokens en un string limpio
    texto_limpio = ' '.join(tokens)
    return texto_limpio


# Aplicar la función de limpieza a la columna 'noticia' de un DataFrame
# Asumiendo que 'df' es un DataFrame y 'noticia' es la columna a procesar
df['noticia'] = df['noticia'].apply(limpiar_texto)

# Mostrar las primeras filas del DataFrame para verificar la limpieza
print(df.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexeieacs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##**Pregunta - 3:**



Encontrar la matriz Tf-idf de la columna de noticias. Despliega los primeros 5 renglones con
algunas de sus columnas con sus nombres, donde las columnas son los tokens. ¿Cuál es el
significado de cada renglón? ¿Y el significado de cada columna?

In [ ]:
# Paso 1: Inicializar el vectorizador TF-IDF
# El vectorizador TF-IDF permite convertir una colección de documentos de texto brutos en una matriz de características TF-IDF.
vectorizer = TfidfVectorizer()

# Paso 2: Ajustar el vectorizador y transformar los datos
# 'fit_transform' aprende el vocabulario del corpus y transforma los datos en una matriz de características documentales.
tfidf_matrix = vectorizer.fit_transform(df['noticia'])

# Paso 3: Convertir la matriz TF-IDF en un DataFrame para una mejor manipulación y visualización
# Utilizamos 'toarray()' para convertir los resultados de la matriz esparsa a una matriz densa.
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),
                        columns=vectorizer.get_feature_names_out())

# Paso 4: Mostrar las primeras filas del DataFrame resultante para verificar y visualizar los datos
# Seleccionamos las primeras 5 filas y las primeras 10 columnas como muestra para entender cómo se representan los datos.
print(tfidf_df.iloc[:5, :10])

## Explicación de la Matriz TF-IDF

### Renglón (Fila)
Cada **renglón** en la matriz TF-IDF representa una **noticia** del conjunto de datos. En el DataFrame resultante, cada renglón contiene los valores TF-IDF de los tokens (palabras) presentes en esa noticia. Estos valores miden tanto la frecuencia de una palabra en una noticia específica, como la importancia de la palabra a lo largo de todo el corpus (conjunto de todas las noticias), ayudando a identificar términos que son distintivamente relevantes en cada noticia.

### Columna
Cada **columna** en la matriz representa un **token** (palabra) del vocabulario generado a partir de todo el corpus de noticias. El valor en una celda específica de la matriz TF-IDF indica la importancia de ese token en la noticia correspondiente. Este valor se calcula utilizando la frecuencia del término (cuántas veces aparece el término en la noticia) ponderada por la frecuencia inversa del documento (cuán raro es el término en todo el corpus de noticias). De esta manera, términos que aparecen frecuentemente en pocas noticias pero son raros en el corpus general reciben una mayor puntuación, destacando su importancia en las noticias donde aparecen.

##**Pregunta - 4:**



Aplica el método de descomposición de valores singulares truncado a la matriz Tf-idf anterior con
10 componentes y obtener el gráfico de la importancia relativa de estas. 

In [ ]:
# Aplicar Truncated SVD
n_components = 10
svd = TruncatedSVD(n_components=n_components)
svd_matrix = svd.fit_transform(tfidf_matrix)

# Obtener la importancia relativa de cada componente
explained_variance = svd.explained_variance_ratio_

# Graficar la importancia relativa de los componentes
plt.figure(figsize=(10, 6))
plt.bar(range(1, n_components + 1),
        explained_variance,
        alpha=0.7,
        align='center')
plt.title('Importancia Relativa de los Componentes SVD')
plt.xlabel('Componentes')
plt.ylabel('Importancia Relativa')
plt.xticks(range(1, n_components + 1))
plt.grid(True)
plt.show()

##**Pregunta - 5:**



Obtener la matriz tokens-temas (term-topic) a partir de la matriz 𝑉
்
 de la descomposición SVD.
Despliega sus primeros 5 renglones donde se incluya el nombre de las columnas. 


In [ ]:
# Obtener la matriz V^T
Vt_matrix = svd.components_

# Convertir la matriz V^T a un DataFrame
terms = vectorizer.get_feature_names_out()
term_topic_df = pd.DataFrame(
    Vt_matrix.T,
    index=terms,
    columns=[f'Tema {i+1}' for i in range(n_components)])

# Mostrar los primeros 5 renglones del DataFrame
print(term_topic_df.head())

##**Pregunta - 6:**



Con base a la cantidad de conceptos latentes que determinaste en el ejercicio anterior, obtener cada
uno de sus gráficos con sus 10 términos/tokens más importantes. ¿Cómo describirías cada uno de
dichos conceptos latentes? ¿Se identifican claramente las temáticas de cada uno de ellos? 

In [ ]:
# Función para graficar los 10 términos más importantes para cada tema
def plot_top_terms_per_topic(term_topic_df, num_terms=10):
    for topic in term_topic_df.columns:
        top_terms = term_topic_df[topic].nlargest(num_terms)
        plt.figure(figsize=(10, 6))
        plt.barh(top_terms.index, top_terms.values, color='skyblue')
        plt.xlabel('Importancia')
        plt.ylabel('Términos')
        plt.title(f'Términos más importantes en {topic}')
        plt.gca().invert_yaxis()
        plt.show()


# Graficar los 10 términos más importantes para cada uno de los 10 temas
plot_top_terms_per_topic(term_topic_df)

# Describir cada uno de los temas
for i, topic in enumerate(term_topic_df.columns, 1):
    top_terms = term_topic_df[topic].nlargest(10).index
    print(f"Tema {i}: {', '.join(top_terms)}")

## Descripción de los Temas del Corpus

### Tema 1: Gestión del Coronavirus
- **Palabras Claves**: coronavirus, gobierno, madrid, casos, pp, personas, españa, datos, covid, pandemia
- **Descripción**: Este tema probablemente aborda la gestión y las respuestas del gobierno y partidos políticos como el PP frente a la pandemia de coronavirus en España, incluyendo estadísticas y casos específicos en Madrid.

### Tema 2: Seguimiento Epidemiológico
- **Palabras Claves**: casos, coronavirus, covid, datos, municipio, evolución, pandemia, gráficos, rebrotes, provincias
- **Descripción**: Este tema se centra en el análisis epidemiológico del COVID-19, presentando datos, gráficos y la evolución de la pandemia a nivel municipal y provincial, incluyendo rebrotes.

### Tema 3: Política en el País Vasco
- **Palabras Claves**: euskadi, pnv, puedes, vasco, urkullu, osakidetza, contenidos, álava, bildu, eh
- **Descripción**: Este tema trata sobre la política en el País Vasco, destacando la actividad de partidos como el PNV y Bildu, y figuras políticas como Urkullu, además de mencionar la sanidad regional (Osakidetza).

### Tema 4: Política Municipal y COVID-19
- **Palabras Claves**: pp, municipio, casado, datos, covid, rebrotes, provincias, mapa, partido, municipios
- **Descripción**: Explora la interacción entre la política municipal, especialmente del partido PP liderado por Casado, con la gestión de la pandemia, incluyendo mapas y datos de rebrotes en varias provincias.

### Tema 5: Caso Villarejo en Municipios
- **Palabras Claves**: villarejo, municipio, policía, comisario, datos, rebrotes, provincias, covid, mapa, juez
- **Descripción**: Discute el impacto y las implicaciones del caso del comisario Villarejo en diversos municipios, incluyendo aspectos judiciales y conexiones con la pandemia.

### Tema 6: Villarejo y la Política Madrileña
- **Palabras Claves**: villarejo, madrid, policía, comisario, ayuso, sanidad, casos, comunidad, díaz, bárcenas
- **Descripción**: Centrado en Madrid, este tema aborda las relaciones entre el comisario Villarejo, políticos locales como Ayuso, y otros escándalos políticos como el caso Bárcenas.

### Tema 7: Aspectos Económicos de la Pandemia
- **Palabras Claves**: euros, madrid, pnv, erte, montai, ayuso, coronavirus, empresas, gobierno, millones
- **Descripción**: Trata sobre las consecuencias económicas de la pandemia en Madrid y otras regiones, incluyendo ERTEs, ayudas a empresas y la gestión financiera del gobierno y partidos como el PNV.

### Tema 8: Educación y Política
- **Palabras Claves**: máster, cifuentes, alumnos, educación, casado, curso, municipio, universidad, centros, profesores
- **Descripción**: Enfocado en la educación, este tema discute controversias como el caso del máster de Cifuentes, implicaciones políticas relacionadas con Casado, y la gestión educativa en universidades y municipios.

### Tema 9: Información y Medios
- **Palabras Claves**: coronavirus, casado, máster, boletín, información, cifuentes, eldiario, hazte, álvarez, rey
- **Descripción**: Analiza cómo los medios y publicaciones como eldiario.es tratan temas de actualidad como el coronavirus, la política, y escándalos educativos, influenciando la opinión pública.

### Tema 10: Realeza y Finanzas
- **Palabras Claves**: casos, rey, millones, carlos, juan, euros, emérito, máster, países, muertes
- **Descripción**: Este tema podría explorar las implic

## **Pregunta - 7:**



Utiliza la librería Gensim para implementar ahora la técnica de LDA. Revisa la documentación
correspondiente y aplica de preferencia el modelo paralelizable:
https://radimrehurek.com/gensim/models/ldamodel.html
https://radimrehurek.com/gensim/models/ldamulticore.html 


In [ ]:
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

# Asegúrate de descargar los recursos de NLTK si aún no lo has hecho
# nltk.download('punkt')
# nltk.download('stopwords')

# Configurar el lematizador y stopwords
stop_words = set(
    stopwords.words('spanish'))  # Cambiar el idioma según sea necesario
stemmer = SpanishStemmer()


# Función para limpiar y preprocesar texto, devolviendo tokens
def limpiar_texto_and_return_tokens(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(
        r'[\W_]+', ' ',
        texto)  # Eliminar caracteres especiales y signos de puntuación
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    tokens = word_tokenize(texto)  # Tokenización
    tokens = [
        stemmer.stem(token) for token in tokens if token not in stop_words
    ]  # Lematización y eliminación de stopwords
    return tokens


# Aplicar la función de limpieza y tokenización a la columna 'noticia'
df['tokens'] = df['noticia'].apply(limpiar_texto_and_return_tokens)

# Crear el diccionario y el corpus de bolsa de palabras
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]

# Configurar y aplicar el modelo LDA Multicore
num_topics = 10
lda_model = LdaMulticore(corpus,
                         num_topics=num_topics,
                         id2word=dictionary,
                         passes=10,
                         workers=multiprocessing.cpu_count() - 1)

# Imprimir los temas identificados por LDA
for idx, topic in lda_model.print_topics(-1):
    print(f'Tema {idx + 1}: {topic}')


# Función para visualizar los términos más importantes por tema
def plot_top_terms_per_topic_lda(lda_model, num_terms=10):
    for i in range(num_topics):
        terms = lda_model.show_topic(i, num_terms)
        terms_df = pd.DataFrame(terms, columns=['Term', 'Weight'])
        plt.figure(figsize=(10, 6))
        plt.barh(terms_df['Term'], terms_df['Weight'], color='skyblue')
        plt.xlabel('Importancia')
        plt.ylabel('Términos')
        plt.title(f'Términos más importantes en Tema {i + 1}')
        plt.gca().invert_yaxis()
        plt.show()


# Llamar a la función de visualización
plot_top_terms_per_topic_lda(lda_model)

## **Pregunta - 8:**



Con base a esta técnica ¿qué cantidad de tópicos consideras que es la más adecuada? Compara tus
resultados con el método LSI. ¿Qué encuentras de coincidencias y diferencias? ¿Cuál consideras
puede ser el mejor resultado, es decir, cuál consideras puede ser la mejor cantidad de tópicos a
considerar?

In [ ]:
# Función para calcular la coherencia de tópicos para diferentes números de temas
def compute_coherence_values(dictionary,
                             corpus,
                             texts,
                             limit,
                             start=2,
                             step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus,
                             num_topics=num_topics,
                             id2word=dictionary,
                             passes=10,
                             workers=multiprocessing.cpu_count() - 1)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model,
                                        texts=texts,
                                        dictionary=dictionary,
                                        coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values


# Calcular la coherencia para diferentes números de temas
limit = 30
start = 2
step = 2
model_list, coherence_values = compute_coherence_values(
    dictionary, corpus, df['tokens'], limit, start, step)

# Graficar los resultados
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Número de Tópicos")
plt.ylabel("Coherencia de Tópicos")
plt.title("Coherencia de Tópicos para LDA")
plt.show()

# Mostrar el número óptimo de tópicos
optimal_num_topics = x[coherence_values.index(max(coherence_values))]
print(f"Número óptimo de tópicos: {optimal_num_topics}")

# Comparar con LSI
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=optimal_num_topics)
lsi_coherence_model = CoherenceModel(model=lsi_model,
                                     texts=df['tokens'],
                                     dictionary=dictionary,
                                     coherence='c_v')
lsi_coherence = lsi_coherence_model.get_coherence()

print(f"Coherencia de Tópicos para LSI: {lsi_coherence}")
print(f"Coherencia de Tópicos para LDA: {max(coherence_values)}")

##### Número Óptimo de Tópicos:

El número óptimo de tópicos se determina basado en la coherencia de los temas. Si la coherencia de LDA es significativamente mayor que la de LSI, LDA puede ser preferible.
##### Coincidencias y Diferencias:

LDA: Tiende a generar temas más coherentes y especializados.
LSI: Puede generar temas más difusos y generales.
##### Mejor Cantidad de Tópicos:

Basado en la coherencia, la cantidad de tópicos óptima sería aquella que maximiza la coherencia en la gráfica de LDA.
Si la coherencia de LDA es mayor que la de LSI, entonces LDA con el número óptimo de tópicos es preferible.

##**Pregunta - 9:**



Incluye tus conclusiones finales de la actividad. 